# <a id="table-of-contents"></a>Table of Contents

1. [Intraday Mean Reversion](#section1)
2. [Moving Average](#section2)


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
import seaborn as sns


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import precision_recall_curve, confusion_matrix, ConfusionMatrixDisplay,  roc_curve, roc_auc_score, average_precision_score


from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from arch import arch_model
from statsmodels.stats.stattools import durbin_watson

In [ ]:
# Define the path to the Downloads folder and the file
downloads_folder = os.path.expanduser("~/Desktop")
file_name = "aaHistoricalData_1726248252859.csv" 
file_path = os.path.join(downloads_folder, file_name)

# Read the CSV file into a DataFrame
data = pd.read_csv(file_path)

In [ ]:
# Convert 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Sort the DataFrame by date in ascending order
data = data.sort_values(by='Date', ascending=True)

print(data.head(3))

In [ ]:
# Assuming 'data' is your existing DataFrame
data['Original_Index'] = data.index  # Add the Original_Index column to capture current index
print('''It will record the INDEX to make sure that future data manipulations don't force us to lose the order.''')

In [ ]:
# Add new columns with the given calculations
data['High_minus_Open'] = (data['High'] - data['Open']) / ((data['Open'] + data['Close/Last']) / 2)
data['day_change'] = (data['Close/Last'] - data['Close/Last'].shift(1)) / data['Close/Last'].shift(1)

data['opening_gaps'] = (data['Open'] - data['Close/Last'].shift(1)) / ((data['Open'] + data['Close/Last']) / 2)

#YOU DEFINED OPENNING GAPS AS TODAY'S OPEN TO YESTERDAY'S CLOSE INSTEAD OF TODAY'S OPEN TO YESTERDAY'S HIGH/LOW..


data['net_Intraday_Movement_Around_Open_Ratio'] = ((data['High'] - data['Open']) / ((data['Open'] + data['Close/Last']) / 2)) - ((data['Open'] - data['Low']) / ((data['Open'] + data['Close/Last']) / 2))
data['Intraday_Volatility'] = ((data['High'] - data['Open']) / ((data['Open'] + data['Close/Last']) / 2)) + ((data['Open'] - data['Low']) / ((data['Open'] + data['Close/Last']) / 2))

# Calculate day change adjusted for opening gap
data['day_change_adjusted_for_openning_gap'] = data['day_change'] - data['opening_gaps']


# Create moving average columns
data['MA_5'] = data['Open'].rolling(window=5).mean()
data['MA_14'] = data['Open'].rolling(window=14).mean()
data['MA_50'] = data['Open'].rolling(window=50).mean()
data['MA_200'] = data['Open'].rolling(window=200).mean()


# Create lagged features comparing today's open to these moving averages
data['open_above_MA_5'] = (data['Open'] > data['MA_5']).astype(int)
data['open_above_MA_14'] = (data['Open'] > data['MA_14']).astype(int)
data['open_above_MA_50'] = (data['Open'] > data['MA_50']).astype(int)

# Create a crossover signal column
data['MA_Crossover_Signal_5_14'] = ((data['MA_5'] > data['MA_14']) & (data['MA_5'].shift(1) <= data['MA_5'].shift(1))).astype(int)
data['MA_Crossover_Signal_today_5'] = ((data['Close/Last'] > data['MA_14']) & (data['Close/Last'].shift(1) <= data['MA_5'].shift(1))).astype(int)


# Create a binary indicator for whether yesterday's close was above or below the 5-day MA
data['yesterday_close_above_MA_5'] = (data['Close/Last'].shift(1) > data['MA_14']).astype(int)


## <a id="section2"></a> Moving Averages: Today's Price Change

In [ ]:
# Create lagged feature comparing today's open to the 200-day moving average
data['open_above_MA_200'] = (data['Open'] > data['MA_200']).astype(int)

# Drop rows with NaN values in the columns of interest and explicitly make a copy
clean_data = data.dropna(subset=['open_above_MA_5', 'open_above_MA_14', 'open_above_MA_50', 'open_above_MA_200', 'day_change']).copy()

# Logistic regression preparation: Convert 'day_change' to binary (0 for Down, 1 for Up)
clean_data['day_change'] = clean_data['day_change'].apply(lambda x: 1 if x > 0 else 0)

# Independent variables (including the new 200-day moving average) and dependent variable (day_change)
X = clean_data[['open_above_MA_5', 'open_above_MA_14', 'open_above_MA_50', 'open_above_MA_200']]
y = clean_data['day_change']

# Add a constant term to the predictor
X = sm.add_constant(X)

# Fit the logistic regression model
model = sm.Logit(y, X)
result = model.fit()

# Print the logistic regression results, including confidence intervals
print(result.summary())

# Extract confidence intervals and rename columns for clarity
conf = result.conf_int()
conf.columns = ['Lower Bound for 95% Interval', 'Upper Bound for 95% Interval']

# Exponentiate coefficients to get odds ratios
conf['Odds Ratio'] = np.exp(result.params)

# Calculate the lower and upper bounds for the odds ratios
conf['Odds Ratio Lower Bound'] = np.exp(conf['Lower Bound for 95% Interval'])
conf['Odds Ratio Upper Bound'] = np.exp(conf['Upper Bound for 95% Interval'])

# Display the confidence intervals and odds ratios
print(conf)

# Calculate probabilities from odds ratios
odds_ratios = np.exp(result.params)
probabilities = odds_ratios / (1 + odds_ratios)

# Calculate the lower and upper bounds for the probabilities
prob_lower_bound = conf['Odds Ratio Lower Bound'] / (1 + conf['Odds Ratio Lower Bound'])
prob_upper_bound = conf['Odds Ratio Upper Bound'] / (1 + conf['Odds Ratio Upper Bound'])

# Display the probabilities and their confidence intervals
prob_conf = pd.DataFrame({
    'Probability': probabilities,
    'Lower Bound for 95% Interval': prob_lower_bound,
    'Upper Bound for 95% Interval': prob_upper_bound
})

print("\nProbabilities with Confidence Intervals:")
print(prob_conf)


In [ ]:
# Plot ACF and PACF for the dependent variable 'day_change'
plt.figure(figsize=(12, 6))

plt.subplot(121)
plot_acf(clean_data['day_change'], lags=30, ax=plt.gca())
plt.title('ACF of Day Change')

plt.subplot(122)
plot_pacf(clean_data['day_change'], lags=30, ax=plt.gca())
plt.title('PACF of Day Change')

plt.show()

print('''
The below charts show that there are no temporal dependencies for the dependent variables with itself.
''')


In [ ]:
# Predict probabilities
y_pred_prob = result.predict(X)
y_pred = (y_pred_prob >= 0.5).astype(int)

# Confusion Matrix
cm = confusion_matrix(y, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

# ROC Curve and AUC
fpr, tpr, _ = roc_curve(y, y_pred_prob)
auc_score = roc_auc_score(y, y_pred_prob)
plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y, y_pred_prob)
average_precision = average_precision_score(y, y_pred_prob)
plt.figure()
plt.plot(recall, precision, label=f'Precision-Recall curve (AP = {average_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.show()


In [ ]:
# Create new columns for visualization
clean_data['combined_label'] = clean_data.apply(
    lambda row: f"5MA: {'Above' if row['open_above_MA_5'] else 'Below'} / "
                f"14MA: {'Above' if row['open_above_MA_14'] else 'Below'} / "
                f"50MA: {'Above' if row['open_above_MA_50'] else 'Below'} / "
                f"200MA: {'Above' if row['open_above_MA_200'] else 'Below'}", axis=1)
clean_data['day_change_label'] = clean_data['day_change'].apply(lambda x: 'Up' if x == 1 else 'Down')

# Count the occurrences of each combination of combined_label and 'day_change_label'
counts = clean_data.groupby(['combined_label', 'day_change_label']).size().unstack().fillna(0)

# Normalize the counts to get proportions
proportions = counts.div(counts.sum(axis=1), axis=0)

# Calculate the absolute difference between 'Up' and 'Down' proportions for sorting
proportions['Difference'] = abs(proportions['Up'] - proportions['Down'])
proportions = proportions.sort_values('Difference', ascending=False).drop(columns='Difference')

# Create the stacked bar chart
ax = proportions.plot(kind='bar', stacked=True, color=['red', 'green'], figsize=(12, 8))

# Add percentages on top of the bars
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    ax.annotate(f'{height:.0%}', (x + width / 2, y + height / 2), ha='center', va='center', color='white', fontsize=12, fontweight='bold')

# Add labels and title
plt.xlabel('Open Status Relative to Moving Averages')
plt.ylabel('Proportion')
plt.title('Proportion of Day Change (Up/Down) vs Open Status Relative to Moving Averages')

# Add legend
plt.legend(title='Day Change')

# Show the plot
plt.show()


In [ ]:
# Loop over each moving average period from 1 to 50 days
for i in range(1, 51):
    ma_column = f'open_above_MA_{i}'
    data[ma_column] = data['Open'].rolling(window=i).mean()

# Calculate the binary variables to check if the opening price is above the moving averages
for i in range(1, 51):
    ma_column = f'open_above_MA_{i}'
    data[ma_column] = (data['Open'] > data[ma_column]).astype(int)

# Drop rows with NaN values in the newly created columns
columns_of_interest = [f'open_above_MA_{i}' for i in range(1, 51)]
data.dropna(subset=columns_of_interest, inplace=True)



In [ ]:
# Drop rows with NaN values in the columns of interest and explicitly make a copy
clean_data = data.copy()

# Logistic regression preparation: Convert 'day_change' to binary (0 for Down, 1 for Up)
clean_data['day_change'] = clean_data['day_change'].apply(lambda x: 1 if x > 0 else 0)

# Ensure all columns are numeric
clean_data = clean_data.apply(pd.to_numeric, errors='coerce')

# Convert boolean columns to integers
for i in range(2, 51):
    ma_column = f'open_above_MA_{i}'
    if ma_column in clean_data.columns:
        clean_data[ma_column] = clean_data[ma_column].astype(int)

# Drop rows with NaN values in the 'day_change' column and moving average columns
columns_of_interest = ['day_change'] + [f'open_above_MA_{i}' for i in range(2, 51)]
clean_data = clean_data.dropna(subset=columns_of_interest)

# Prepare a list to collect results
results_list = []

# Loop over each moving average from 2 to 50 days
for i in range(2, 51):
    ma_column = f'open_above_MA_{i}'
    if ma_column in clean_data.columns:
        X = clean_data[[ma_column]]
        y = clean_data['day_change']
        
        # Add a constant term to the predictor
        X = sm.add_constant(X)
        
        # Fit the logistic regression model
        model = sm.Logit(y, X)
        result = model.fit(disp=0)  # disp=0 suppresses the output
        
        # Extract coefficient, p-value, and Durbin-Watson statistic
        coef = result.params[ma_column]
        p_value = result.pvalues[ma_column]
        dw_stat = durbin_watson(result.resid_generalized)
        
        # Only collect significant results with beta > 0.4
        if p_value < 0.05 and coef > 0.4:
            odds_ratio = np.exp(coef)
            prob = odds_ratio / (1 + odds_ratio)
            
            # Collect the results
            results_list.append({
                'Moving Average': i,
                'Coefficient': coef,
                'P-Value': p_value,
                'Odds Ratio': odds_ratio,
                'Probability': prob,
                'Durbin-Watson': dw_stat
            })

# Convert results list to a DataFrame
results_summary = pd.DataFrame(results_list)

# Calculate confidence intervals for odds ratios and probabilities
results_summary['Odds Ratio Lower Bound'] = np.exp(results_summary['Coefficient'] - 1.96 * results_summary['Coefficient'].std())
results_summary['Odds Ratio Upper Bound'] = np.exp(results_summary['Coefficient'] + 1.96 * results_summary['Coefficient'].std())
results_summary['Probability Lower Bound'] = results_summary['Odds Ratio Lower Bound'] / (1 + results_summary['Odds Ratio Lower Bound'])
results_summary['Probability Upper Bound'] = results_summary['Odds Ratio Upper Bound'] / (1 + results_summary['Odds Ratio Upper Bound'])

# Sort results by coefficient for better readability
results_summary_sorted = results_summary.sort_values(by='Coefficient', ascending=False)

# Print the results
print("\nSignificant Moving Averages with Coefficients > 0.4:")
print(results_summary_sorted)

# Plot stacked bar graph of probabilities with confidence intervals
plt.figure(figsize=(12, 8))
plt.bar(results_summary_sorted['Moving Average'], results_summary_sorted['Probability'], yerr=[results_summary_sorted['Probability'] - results_summary_sorted['Probability Lower Bound'], results_summary_sorted['Probability Upper Bound'] - results_summary_sorted['Probability']], capsize=5, color='skyblue')
plt.xlabel('Moving Average Period (Days)')
plt.ylabel('Probability')
plt.title('Probability of Next Day\'s Price Movement Being Positive')
plt.xticks(ticks=np.arange(0, 20, 2), labels=np.arange(0, 20, 2))
plt.axhline(y=0.5, color='red', linestyle='--')
plt.show()

print('''
The logistic regression analysis confirms that shorter-term moving averages are significant predictors of the next day's price movement. Next steps would be
to determine if there is any information in the later moving averages.. say 14 that is not included or already encapsulated in the 5 day.
''')

print(''' 
You're correct that multicollinearity is not a concern when using a single moving average as the predictor. Your approach to isolate and select the most 
predictive moving average is sound.
''')

print('''
Durbin-Watson confirms that there is no autocorrelation
''')


In [ ]:
# Plot ACF and PACF for the dependent variable
plt.figure(figsize=(12, 6))

plt.subplot(121)
plot_acf(clean_data['day_change'], lags=30, ax=plt.gca())
plt.title('ACF of Day Change')

plt.subplot(122)
plot_pacf(clean_data['day_change'], lags=30, ax=plt.gca())
plt.title('PACF of Day Change')

plt.show()

print('''
the below charts show that there is no temporal dependencies for the dependent vairables with itself
''')


In [ ]:
# Drop rows with NaN values in the columns of interest and explicitly make a copy
clean_data = data.copy()

# Logistic regression preparation: Convert 'day_change' to binary (0 for Down, 1 for Up)
clean_data['day_change'] = clean_data['day_change'].apply(lambda x: 1 if x > 0 else 0)

# Ensure all columns are numeric
clean_data = clean_data.apply(pd.to_numeric, errors='coerce')

# Convert boolean columns to integers
for i in range(2, 51):
    ma_column = f'open_above_MA_{i}'
    if ma_column in clean_data.columns:
        clean_data[ma_column] = clean_data[ma_column].astype(int)

# Drop rows with NaN values in the 'day_change' column and moving average columns
columns_of_interest = ['day_change'] + [f'open_above_MA_{i}' for i in range(2, 51)]
clean_data = clean_data.dropna(subset=columns_of_interest)

# Ensure the dependent variable is binary
assert clean_data['day_change'].isin([0, 1]).all(), "Dependent variable must be binary."

# Function to fit logistic regression and plot evaluation metrics
def evaluate_moving_average(ma_column):
    X = clean_data[[ma_column]]
    y = clean_data['day_change']
    
    # Add a constant term to the predictor
    X = sm.add_constant(X)
    
    # Fit the logistic regression model
    model = sm.Logit(y, X)
    result = model.fit(disp=0)  # disp=0 suppresses the output
    
    # Predict probabilities and classes
    y_pred_prob = result.predict(X)
    y_pred = (y_pred_prob >= 0.5).astype(int)
    
    # Confusion Matrix
    cm = confusion_matrix(y, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix for {ma_column}')
    plt.show()
    
    # ROC Curve and AUC
    fpr, tpr, _ = roc_curve(y, y_pred_prob)
    auc_score = roc_auc_score(y, y_pred_prob)
    plt.figure()
    plt.plot(fpr, tpr, label=f'ROC curve (AUC = {auc_score:.2f}) - {ma_column}')
    plt.plot([0, 1], [0, 1], color='red', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for {ma_column}')
    plt.legend(loc="lower right")
    plt.show()
    
    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y, y_pred_prob)
    average_precision = average_precision_score(y, y_pred_prob)
    plt.figure()
    plt.plot(recall, precision, label=f'Precision-Recall curve (AP = {average_precision:.2f}) - {ma_column}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve for {ma_column}')
    plt.legend(loc="lower left")
    plt.show()

# Evaluate for MA_2 and MA_3
evaluate_moving_average('open_above_MA_2')
evaluate_moving_average('open_above_MA_3')


In [ ]:
# Drop rows with NaN values in the columns of interest
clean_data = data.dropna(subset=['open_above_MA_5', 'open_above_MA_14', 'open_above_MA_50', 'day_change'])

# Independent variables
X = clean_data[['open_above_MA_5', 'open_above_MA_14', 'open_above_MA_50']]

# Add a constant term to the predictor
X = sm.add_constant(X)

# Initialize a DataFrame to hold VIF and R^2 values
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

# Calculate VIF and R^2 for each variable
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data["R_squared"] = [
    sm.OLS(X.iloc[:, i], X.iloc[:, [j for j in range(X.shape[1]) if j != i]]).fit().rsquared for i in range(X.shape[1])
]

print(vif_data)


In [ ]:
print('''

Exactly. The low VIF values indicate that these moving averages are not strongly related to each other in predicting daily changes. This reinforces the 
idea that the data lacks a significant trend within the 5, 14, and 50-day windows. It suggests a level of randomness, with no clear pattern or momentum 
over these periods. So, the moving averages are not =trending indicators in this context. What's your next data adventure?

''')

In [ ]:
# Initialize the column to store the sum of moving averages
data['Sum_MA_1_50'] = 0

# Calculate and sum the moving averages from 1 to 50 days
for i in range(1, 51):
    # Calculate the moving average for each window size 'i'
    ma = data['Close/Last'].rolling(window=i).mean()
    # Add the calculated moving average to 'Sum_MA_1_50'
    data['Sum_MA_1_50'] += ma

# Divide by 50 to get the average
data['Sum_MA_1_50_Avg'] = data['Sum_MA_1_50'] / 50

# Ensure 'day_change' is defined
data['day_change'] = (data['Close/Last'] - data['Close/Last'].shift(1)) / data['Close/Last'].shift(1)

# Drop rows with NaN values in the columns of interest
clean_data = data.dropna(subset=['Sum_MA_1_50_Avg', 'day_change'])

# Independent variable and dependent variable
X = clean_data[['Sum_MA_1_50_Avg']]
y = clean_data['day_change']

# Add a constant term to the predictor
X = sm.add_constant(X)

# Fit the linear regression model
model = sm.OLS(y, X).fit()

# Print the OLS results
print(model.summary())

# Predict values
predictions = model.predict(X)

# Create the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(clean_data['Sum_MA_1_50_Avg'], clean_data['day_change'], color='blue', alpha=0.1, label='Data points')

# Plot the regression line
plt.plot(clean_data['Sum_MA_1_50_Avg'], predictions, color='black', linewidth=2, label='Regression Line')

# Add labels and title
plt.xlabel('Sum of Moving Averages (1-50 days) Averaged')
plt.ylabel('Day Change')
plt.title('Regression of Day Change on Sum of Moving Averages (1-50 days) Averaged')

# Set the x-axis limit to fit the range of 'Sum_MA_1_50_Avg'
plt.xlim(clean_data['Sum_MA_1_50_Avg'].min(), clean_data['Sum_MA_1_50_Avg'].max())

# Add legend
plt.legend()

# Show and save the plot
plt.show()

print('''

Exactly. The OLS results show that the average of the MAs isn't predictive for day-to-day price changes. The small coefficient, 
high p-value, and low R-squared all suggest the sum of moving averages isn't a useful predictor.

This finding aligns with your earlier insights about the lack of significance in individual moving averages. So, it seems like moving
averages, at least in these configurations, don't offer predictive power for daily changes.

''')
